In [21]:
from pathlib import Path
import os
import random
import shutil

im_dir = "Images/"  # whereever you want to get files from
dst_dir = "ImagesSubSet"  # destination folder
paths = list(Path(im_dir).glob('**/*.jpg'))
sample = random.sample(paths, 300)
random.shuffle(sample)
set_a_paths = sample[:100]
set_b_paths = sample[100:200]
set_c_paths = sample[200:250]
set_d_paths = sample[250:]

for set, paths in zip(['A','B','C','D'],[set_a_paths,set_b_paths,set_c_paths,set_d_paths]):
    for src in paths:
        path = os.path.join(dst_dir + set, src.name)
        os.makedirs(os.path.dirname(path), exist_ok=True)
        shutil.copy(src, path)